# **installing Modules**

In [1]:
!pip install fpdf

In [2]:
!pip install telebot

In [3]:
!pip install pyTelegramBotAPI

# **Modules**

In [1]:
import pandas as pd
import nltk
import numpy as np
import string
import warnings
import requests
import pickle
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# *Replies***

In [2]:

data = {
        0 : ["Yes"],
        1 :  ["No"],
        2 :  ["Cancel Order"],
        3 :   ["Res Turant Foods is a multicuisine restaurant with user ratings : 4.5 here you find all yours favourite dishes we provide high quality food to our cutomers at affordable price "],
        4 :   ["Our rating is 4.5"],
        5 :   ["if i recommend you some dishes than it should be cheese pakoda,veg panner tikka,veg maharaja","according to me you can try cheese pakoda yumm😋","try veg panner tikka 🤤" ,"some special dishes are veg panner tikka ,veg maharaja missal"],
        6 :   ["yeah true food is secret of energy","yeah so true"],
        7:    ["hello","hey","Hello Welcome","Welcome to Resturant Foods"],
        8 :  ["order"],
        9:  ["yeah Great "]
       
}

In [3]:
para = """  Good Morning 🌞.
            GoodMorning 🌞.
            Good Afternoon 🌞 .
            GoodAfternoon 🌞 .
            Good Night 🌚 .
            GoodNight 🌚 .
            GoodEvening .
            Good Evening .
            My Creator is Arpit .
            
          """


# **Text processing**

In [4]:
para=para.lower()

In [5]:
sent_tokens = nltk.sent_tokenize(para)
word_tokens = nltk.word_tokenize(para)

In [6]:
lemmer = nltk.stem.WordNetLemmatizer()

In [7]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [8]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [9]:
def Normalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    

# **Model Training**

In [10]:
df = pd.read_csv("botmad.csv")
df.head()

,label,class
0,Yes I would like to order,0
1,yes I'm very hungry,0
2,yep,0
3,y,0
4,ye,0


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
x = df['label']
y = df['class']

In [13]:
Vec = TfidfVectorizer(tokenizer=Normalize,stop_words = 'english')

In [14]:
feature = Vec.fit_transform(x)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [15]:
clf  = RandomForestClassifier(n_estimators = 30)

In [16]:
clf.fit(feature,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
test = ["No"]
a = clf.predict_proba(Vec.transform(test))[0]

# **Generating Bill**

In [18]:
from datetime import date

today = date.today()


In [19]:
menu = pd.read_csv("menu.csv")
menu['i'] = menu['id']
menu = menu.set_index('id')
menu.head()

,name,Unnamed: 2,rate,i
id,,,,
1,Ussal,NaN,25,1
2,Missal,NaN,30,2
3,Dahi Missal,NaN,40,3
4,Pav,NaN,5,4
5,Garam Pav,NaN,6,5


In [20]:
a = str(menu[menu['i'] == 1]).split("\n")[2].split()
a[1:-3]

['Ussal']

In [21]:
def getbill(detail):
      s = create_receipt(detail[0]['name'],detail[1]['contact'],detail[2]['orders'])
      bot.send_document(from_,open('bill.pdf', 'rb'));
      share = 0
      detail = []
      tbot.send_message("thank you for ordering food your order is on the way this is your bill of amount "+str(s),from_)
      
def create_receipt(name,contact,orders):
      d = date.today()
      s = 0
      f = open("bill.txt","w")
      f.write("""\n\n\tRes turant Foods\n""")
      f.write("""\t-------------------\t\t\n\n\t\tINVOICE\t\t\n\n""")
      f.write("Date : "+str(d)+"\n")
      f.write("Name : "+name+"\t\t Mob no :"+contact+"\n")
      for i in orders:
        try:
          i =  int(i)
        except:
          continue
        print("i === ",type(i),i,orders)
        order = str(menu[menu['i'] == i]).split("\n")[2].split()
        s = s + int(order[-2])
        f.write("""\t"""+" ".join(order[1:-3])+"""   """+order[-2]+"""\n""")
      f.write("""-----------------------\nTotal Bill \t\t"""+str(s))
      f.close()
      from fpdf import FPDF 
      pdf = FPDF()      
      pdf.add_page() 
      pdf.set_font("Arial", size = 15) 
      f = open("bill.txt", "r") 
      for x in f: 
        pdf.cell(200, 10, txt = x, ln = 1, align = 'C') 
      pdf.output("bill.pdf")  
      return s

# **Creating Response**

In [22]:
import random
def response(user_response):
      robo_response=''
      sent_tokens.append(user_response)
      TfidfVec = TfidfVectorizer(tokenizer=Normalize,stop_words = 'english')
      tfidf = TfidfVec.fit_transform(sent_tokens)
      vals = cosine_similarity(tfidf[-1], tfidf)  
      idx=vals.argsort()[0][-2]
      flat = vals.flatten() 
      flat.sort()
      req_tfidf = flat[-2]
      if(req_tfidf==0):
          pred = clf.predict_proba(Vec.transform([user_response]))[0]
          a = pred.argsort()[-1]
          if pred[a] >= 0.6:
            print(a,"\n\n",random.choice(data[a]))
            return random.choice(data[a])
          else:
            ran = ["sorry , i don't know what is that ","sorry, i didn't understand this","Sorry for inconvience i'm not able to understand","please try other ques i'm not able to understand this"]
            robo_response=robo_response+random.choice(ran)
            return robo_response
      else:
          robo_response = robo_response + sent_tokens[idx]
          return robo_response


# **Accesing Bot**

In [23]:
import json
token = "1123219618:AAEZI91C-EHuJPUoTwZvR2R2W0Vg3YWGnKI"

class telegram_bot():
    def __init__(self):
        self.token=token 
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)
    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

In [24]:
import telebot
bot = telebot.TeleBot(token)

# **Creating Markups**

In [25]:
from telebot import types

#yes No Markups 
markup = types.ReplyKeyboardMarkup()
markup.add('Yes', 'No')

#markup share contact
contact_keyboard = types.KeyboardButton(text="send_contact", request_contact=True)
reply_markup = types.ReplyKeyboardMarkup()
reply_markup.add(contact_keyboard)

#cancel
cancel = types.ReplyKeyboardMarkup()
cancel.add('Cancel Order')

#none markup
m = types.ReplyKeyboardMarkup()
m.add('No')



# **Deploying bot on telegram and make reply**

In [27]:
tbot = telegram_bot()
update_id = None
def make_reply(name,msg):
    if msg is not None:
        flag=True
        if msg == '/start':
          return "hello " + name + " Welocme to resturant foods Want to order write order"
        while(flag==True):
            user_response = msg
            user_response=user_response.lower()
            if(user_response!='bye'):
                if(user_response=='thanks' or user_response=='thank you' ):
                    flag=False
                    return "Welcome 😇 "
                else:
                    r =  response(user_response)
                    try:
                      sent_tokens.remove(user_response)
                    except: 
                      pass      
                    return r
            else:
                flag=False
                bye = ["Bye Byee !!","Bye Thank you for contacting us","Bye"]
                return random.choice(bye)


share = 0 
detail = []  
message = ''
reply = ''
while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    name = updates[0]['message']['from']['first_name']
    print(updates)
    m = message
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)


            try :
                reply = make_reply(name,message)
            except:
              reply = ""
 
         
            


            if reply == "order" or message == "order":
              share = 1
              detail = []
              detail.append({'name':name})
              bot.send_message(from_, text = "Hello "+name+" would u like to order something Press no if you mistakely come in this menu", reply_markup=markup)

    
            elif share == 1:
                if reply == 'Yes' or message == 'Yes':
                  share = 2
                  bot.send_message(chat_id=from_, text="Can you plz share your Contact number", reply_markup=reply_markup)

            elif share == 2:
                share = 3
                m = item['message']['contact']['phone_number']
                detail.append({'contact':m})
                bot.send_photo(chat_id=from_, photo=open('menu.jpg', 'rb'))
                bot.send_message(chat_id = from_,text = "Enter the serial number of digit you want to order like 2,5,6",reply_markup = cancel)

            elif share == 3:
                share = 4
                if reply  == "Cancel Order" and message == "Cancel Order":
                  share = 0
                  bot.send_message(chat_id = from_,text = "order cancel")
                else:
                  detail.append({'orders' :message.split(',')})
                  bot.send_message(chat_id = from_,text="Want to order Something else",reply_markup = markup)

            elif share == 4:
                print("message ==== ",message)
                share = 7
                if reply  == "Yes" or message == "Yes":
                      tbot.send_message("ok send another serial numbers",from_)
                if reply == "No" or message == "No":
                  getbill(detail)


     
            elif share == 7:
                  for i in message.split(','):
                      detail[-1]['orders'].append(i) 
                  share = 6
                  getbill(detail)


            elif message == "Cancel Order" or reply == "Cancel Order":
              bot.send_message(chat_id = from_,text="order cancelled")
            elif message == "No" or reply == "No":
              bot.send_message(chat_id = from_,text = "👍")
            
 

            
            elif reply not in [data[0][0],data[1][0],data[2][0],data[8][0],"",None] :
                print("\n\n share ====",share,"\n\n")
                try:
                  tbot.send_message(reply,from_)
                except:
                  print("r ======",reply)
                continue
            

           


...
[{'update_id': 128052590, 'message': {'message_id': 747, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603773, 'text': 'wants to order'}}, {'update_id': 128052591, 'message': {'message_id': 748, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603781, 'text': 'i want to order food'}}]
128052590
wants to order
976521729
8 

 order


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


128052591
i want to order food
976521729
0 

 Yes


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...
[{'update_id': 128052592, 'message': {'message_id': 751, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603798, 'reply_to_message': {'message_id': 750, 'from': {'id': 1123219618, 'is_bot': True, 'first_name': 'Resturants_Food😋_bot', 'username': 'corona_virus_information_bot'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603793, 'text': 'Can you plz share your Contact number'}, 'contact': {'phone_number': '+916397365442', 'first_name': 'Arpit', 'last_name': 'Chauhan', 'user_id': 976521729}}}]
128052592
976521729
...
[{'update_id': 128052593, 'message': {'message_id': 754, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'user

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...
[{'update_id': 128052594, 'message': {'message_id': 756, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603823, 'text': "no that's sufficient"}}]
128052594
no that's sufficient
976521729
1 

 No
message ====  no that's sufficient
i ===  <class 'int'> 3 ['3', '45']
i ===  <class 'int'> 45 ['3', '45']


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...
[{'update_id': 128052595, 'message': {'message_id': 759, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603840, 'text': 'thanks'}}]
128052595
thanks
976521729
i ===  <class 'int'> 3 ['3', '45', 'thanks']
i ===  <class 'int'> 45 ['3', '45', 'thanks']
...
[{'update_id': 128052596, 'message': {'message_id': 762, 'from': {'id': 976521729, 'is_bot': False, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'language_code': 'en'}, 'chat': {'id': 976521729, 'first_name': 'Arpit', 'last_name': 'Chauhan', 'username': 'arpit09chauhan', 'type': 'private'}, 'date': 1592603848, 'text': 'thank you'}}]
128052596
thank you
976521729


 share ==== 6 


...
[{'update_id': 128052597, 'message': {'message_id': 764, 'from': {'id': 976521729, 'is_bot

IndexError: ignored